# "It's about time to take your medication!"

## or how to write a friendly reminder bot ;-)

#### [https://github.com/blue-yonder/medbot/medbot.slides.html](https://github.com/blue-yonder/medbot/medbot.slides.html)